라이브러리 불러오기

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import pandas as pd
import copy
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from random import randrange
from sklearn.metrics import f1_score
from resnet import resnet50
from sklearn.model_selection import train_test_split

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0' #cudu 설정

데이터 셋 불러오기

In [ ]:
df = pd.read_csv("dataset/gender_age_detection.csv")

x = df["img_path"]
y = df[["age", "gender"]]

train, test 데이터셋 분리하기

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

x_train = x_train.values.tolist()
x_test = x_test.values.tolist()

age = ["age"]
age_train = y_train[age]
age_nodes = age_train.shape[1]
age_train = age_train.values.tolist()

gender = ["gender"]
gender_train = y_train[gender]
gender_nodes = gender_train.shape[1]
gender_train = gender_train.values.tolist()

age = ["age"]
age_test = y_test[age]
age_nodes = age_test.shape[1]
age_test = age_test.values.tolist()

gender = ["gender"]
gender_test = y_test[gender]
gender_nodes = gender_test.shape[1]
gender_test = gender_test.values.tolist()

Custom_dataset 만들기

In [ ]:
class Custom_dataset(Dataset):
    def __init__(self, image_lists, transform=None):

        self.image_lists = image_lists
        self.transform = transform

    def __getitem__(self, index):
        # random_index = randrange(len(self.king_of_lists[index]))

        img1 = Image.open(self.image_lists[0][index])
        img1 = img1.convert('RGB')

        age = self.image_lists[1][index]  # region
        gender = self.image_lists[2][index]  # gender
        if self.transform is not None:
            img1 = self.transform(img1)

        list_of_labels = [torch.from_numpy(np.array(age)),
                          torch.from_numpy(np.array(gender))]

        return img1, list_of_labels[0], list_of_labels[1]

    def __len__(self):
        return len(self.image_lists[0])

data_transform

In [ ]:
batch_size = 32 #배치사이즈 설정

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((227,227)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}  

dataloaders

In [ ]:
train_lists = [x_train, age_train, gender_train]
test_lists = [x_test, age_test, gender_test]

training_dataset = Custom_dataset(image_lists=train_lists, transform=data_transforms['train'] )

test_dataset = Custom_dataset(image_lists=test_lists, transform=data_transforms['val'] )

dataloaders_dict ={'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
                   'val': torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
                    }

dataset_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
criterion = [nn.CrossEntropyLoss(), nn.BCEWithLogitsLoss()] #loss 설정

epoch 옵션 주기 및 best_acc 뽑기

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model_ft.train()  # Set model to training mode
            else:
                model_ft.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            age_corrects = 0.0
            gender_corrects = 0.0

            # Iterate over data.
            for i, (inputs, age, gen) in enumerate(dataloaders_dict[phase]):
            # dkfdkqhrl
                inputs = inputs.to(device)
                age = age.to(device)
                gen = gen.to(device)
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model_ft(inputs)
                #_, preds = torch.max(outputs, 1)
                    loss0 = criterion[0](outputs[0], torch.max(age, 1)[0])
                #loss0 = criterion[0](outputs[0], age)
                    loss1 = criterion[1](outputs[1], gen.float())


                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss = loss0 + loss1
                        print(f'i : {i} / {int((dataset_sizes[phase] / batch_size))}, loss : {loss.item()}')
                        loss.backward()
                        optimizer_ft.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                age_corrects += torch.sum(torch.max(outputs[0], 1)[0] == torch.max(age, 1)[0])
                #age_corrects += torch.sum(torch.max(outputs[0], 1)[1] == age.float())
                gender_corrects += torch.sum(torch.round(outputs[1]) == gen.float())
                #gender_corrects.append(float((np.rint(outputs[1].cpu().detach().numpy()) == gen.float()).sum()))


                # color_corrects += torch.sum(outputs[4] == color.float())
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            age_acc = age_corrects.double() / dataset_sizes[phase]
            gender_acc = gender_corrects.double() / dataset_sizes[phase]
        #age_acc = age_corrects / dataset_sizes[phase]
        #gender_acc = gender_corrects / dataset_sizes[phase]
            print('===================================================================')
            print('phase : {} epoch : {} age_loss : {} gender_loss : {} total_loss : {}'.format(phase, epoch, loss0, loss1, epoch_loss))
            print('phase : {} age_acc : {} / gender_acc : {}'.format(phase, age_acc, gender_acc))
            print('===================================================================')


            current_acc = (age_acc + gender_acc) / 2.

            # deep copy the model
            if phase == 'val' and best_acc < current_acc:
                print('saving with loss of {}'.format(epoch_loss),
                    'improved over previous {}'.format(best_acc))
                best_acc = current_acc
                best_model_wts = copy.deepcopy(model_ft.state_dict())
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(float(best_acc)))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

하이퍼 파라미터 튜닝 및 학습

In [ ]:
model_ft = resnet50() #모델 불러오기
model_ft.to(device)
print(model_ft)

optimizer_grouped_parameters = [
    {'params': [p for n, p in model_ft.named_parameters()]}
]
optimizer_ft = optim.Adam(optimizer_grouped_parameters, lr = 0.0001) #옵티마이저 값 주기
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

model_ft1 = train_model(model_ft, criterion, optimizer_ft, scheduler, num_epochs=50)
torch.save(model_ft1.state_dict(), 'model.pt')
